In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf 
from tensorflow import keras
from keras import Input
from keras import layers
import utils
import rbf_layer
from scipy.spatial import distance

# ---------------------------------------------------------------------------- #
#                                   read data                                  #
# ---------------------------------------------------------------------------- #

# data = "http://lib.stat.cmu.edu/datasets/boston"
data = "boston.csv"
raw_df = pd.read_csv(data, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]
print("Inputs:")
utils.data_summary(X)
print("Outputs:")
utils.data_summary(y)

Inputs:
Shape = (506, 13)
Minimum = 0.0
Maximum = 711.0
Range = 711.0
Variance = 21070.130450709916
Standard Deviation = 145.1555388220164

Outputs:
Shape = (506,)
Minimum = 5.0
Maximum = 50.0
Range = 45.0
Variance = 84.41955615616556
Standard Deviation = 9.188011545278203



In [2]:
# ---------------------------------------------------------------------------- #
#                                normalize data                                #
# ---------------------------------------------------------------------------- #

preprocessing.scale(X, copy=False)
# preprocessing.scale(y, copy=False)
print("Normalized Inputs:")
utils.data_summary(X)

Normalized Inputs:
Shape = (506, 13)
Minimum = -3.9071933049810412
Maximum = 9.933930601860267
Range = 13.841123906841307
Variance = 1.0
Standard Deviation = 1.0



In [3]:
# ---------------------------------------------------------------------------- #
#                      split data to training and testing                      #
# ---------------------------------------------------------------------------- #

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=0)

print("Training Inputs:")
utils.data_summary(X_train)
print("Testing Inputs:")
utils.data_summary(X_test)

print("Training Outputs:")
utils.data_summary(y_train)
print("Testing Outputs:")
utils.data_summary(y_test)

n_train = X_train.shape[0]

Training Inputs:
Shape = (379, 13)
Minimum = -3.9071933049810412
Maximum = 9.933930601860267
Range = 13.841123906841307
Variance = 0.9970688559815751
Standard Deviation = 0.9985333524632891

Testing Inputs:
Shape = (127, 13)
Minimum = -3.8821945679249596
Maximum = 8.136883506576424
Range = 12.019078074501383
Variance = 1.008742396399277
Standard Deviation = 1.004361686047052

Training Outputs:
Shape = (379,)
Minimum = 5.0
Maximum = 50.0
Range = 45.0
Variance = 85.30823553163789
Standard Deviation = 9.236245748768159

Testing Outputs:
Shape = (127,)
Minimum = 5.6
Maximum = 50.0
Range = 44.4
Variance = 81.69901543803087
Standard Deviation = 9.03875076755803



In [4]:
# ---------------------------------------------------------------------------- #
#                          neural network construction                         #
# ---------------------------------------------------------------------------- #

# hidden_size = int(0.1 * n_train)
# hidden_size = int(0.5 * n_train)
hidden_size = int(0.9 * n_train)

model = keras.Sequential()
model.add(Input(shape=(13,)))
model.add(rbf_layer.RBF(hidden_size, utils.InitCentersKMeans(X_train)))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(1, activation='relu'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rbf (RBF)                   (None, 341)               4433      
                                                                 
 dense (Dense)               (None, 128)               43776     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 48,338
Trainable params: 43,905
Non-trainable params: 4,433
_________________________________________________________________


In [5]:
opt = keras.optimizers.SGD(learning_rate=0.001)
model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[utils.root_mean_squared_error, utils.coeff_determination])
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
12/12 [==============================] - 1s 15ms/step - loss: 569.4520 - root_mean_squared_error: 23.7788 - coeff_determination: -6.1806 - val_loss: 525.3027 - val_root_mean_squared_error: 22.9019 - val_coeff_determination: -5.5168
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 503.0359 - root_mean_squared_error: 22.3876 - coeff_determination: -5.2536 - val_loss: 445.1174 - val_root_mean_squared_error: 21.0822 - val_coeff_determination: -4.5232
Epoch 3/100
12/12 [==============================] - 0s 4ms/step - loss: 399.2794 - root_mean_squared_error: 19.7178 - coeff_determination: -4.0188 - val_loss: 317.4472 - val_root_mean_squared_error: 17.8034 - val_coeff_determination: -2.9389
Epoch 4/100
12/12 [==============================] - 0s 4ms/step - loss: 258.3315 - root_mean_squared_error: 15.7759 - coeff_determination: -2.1936 - val_loss: 183.8270 - val_root_mean_squared_error: 13.5435 - val_coeff_determination: -1.2770
Epoch 5/100
12/12 [========

In [ ]:
print(history.params)
# y = model(X_test)
# print(y_test)

# results = model.evaluate(X_train, y_train)
# results = model.evaluate(X_test, y_test)
# print(results)
# predictions = model.predict(X_test[:10])

# print(predictions)
# print(y_test[:10])